### 使用Apriori算法进行关联分析
- 从大规模数据集中寻找物品间的隐含关系被称作关联分析（association analysis）或者关联规则学习（association rule learning）
- 关系可以有两种形式：频繁项集或者关联规则。频繁项集（frequent item sets）是经常出现在一块的物品的集合，关联规则暗示两种物品之间可能存在很强的关系。
- 一个项集的支持度（support）被定义为数据集中包含该项集的记录所占的比例，
- 可信度或置信度（confidence）是针对一条关联规则来定义的。
- 支持度和可信度是用来量化关联分析是否成功的方法。

### Apriori算法
- 优点：易编码实现。
- 缺点：在大数据集上可能较慢。
- 使用数据类型：数值型或者标称型数据。

### Apriori算法的一般过程
（1）收集数据：可以使用任意方法。  
（2）准备数据：任何数据类型都可以，因为我们只保存集合。  
（3）分析数据：可以使用任意方法。  
（4）训练算法：使用Apriori算法来找到频繁项集。  
（5）测试算法：不需要测试过程。  
（6）使用算法：用于发现频繁项集以及物品之间的关联规则。

### Apriori原理
- 如果某个项集是频繁的，那么它的所有子集也是频繁的。反过来，如果一个项集是非频繁的，那么它的所有超集也是非频繁的。
- 使用该原理我们可以避免项集数目的指数增长，从而在合理时间内计算出频繁项集。
- 关联分析的目标包括两项：发现频繁项集和发现关联规则。
- Apriori算法是发现频繁项集的一种方法。

### 数据集扫描的伪代码
对数据集中的每条交易记录tran  
对每个候选项集can：  
    检查一下can是否是tran的子集；  
    如果是，则增加can的计数值  
对每个候选项集：  
如果其支持度不低于最小值，则保留该项集  
返回所有频繁项集列表  

In [3]:
def loadDataSet():
    '''
    创建一个用于测试的简单数据集
    '''
    return [[1,3,4],[2,3,5],[1,2,3,5],[2,5]]

In [4]:
def createC1(dataSet):
    '''
    构建集合C1
    dataSet:数据集
    Returns：
    - 大小为1的所有候选项集的集合
    '''
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item]) #python不能创建只有一个整数的集合，必须使用列表
    C1.sort()
    return list(map(frozenset,C1)) #forazenset是指冰冻的集合，是不可改变的，set做不到
                

In [5]:
def scanD(D, Ck, minSupport):
    '''
    从C1生成L1
    D：数据集
    Ck：候选集合的列表
    minSupport：最小支持度
    Returns：
    - retList：所有满足最小支持度要求的集合
    - supportData：最频繁项集的支持度
    '''
    ssCnt = {}    #创建一个空字典
    for tid in D:    #遍历数据集中的所有交易记录
        for can in Ck:    #遍历Ck中的所有候选集
            if can.issubset(tid):    #如果Ck中的集合是交易记录的一部分
                if not can in ssCnt:ssCnt[can]=1    #若Ck中的集合不包含在字典中，则增加
                else: ssCnt[can] +=1    #若包含，则增加字典中对应的计数值
    numItems = float(len(D))    #获取交易记录总数
    retList = []    #构建一个空列表，存储所有满足最小支持度要求的集合
    supportData = {}    #最频繁项集的支持度
    for key in ssCnt:    #遍历字典中的每个元素并计算支持度
        support = ssCnt[key]/numItems    #计算支持度
        if support >= minSupport:    #若满足最小支持度要求
            retList.insert(0,key)    #存储在列表中
        supportData[key] = support    #记录支持度
    return retList,supportData    #返回最频繁项集及其支持度

In [6]:
dataSet = loadDataSet()

In [7]:
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [8]:
D = list(map(set,dataSet))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [9]:
L1,suppData0 = scanD(D,C1,0.5)
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [10]:
'''
整个Apriori算法的伪代码：
当集合中项的个数大于0时
    构建一个k个项组成的候选项集的列表
    检查数据以确认每个项集都是频繁的
    保留频繁项集并构建k+1项组成的候选项集的列表
'''
def aprioriGen(Lk,k):    #creates Ck
    '''
    创建候选集Ck
    Lk：频繁项集列表
    k：项集元素个数
    Return:
    - retList:大小为k的所有候选项集的集合
    '''
    retList = []    #创建一个空列表
    lenLk = len(Lk)    #计算Lk中的元素个数
    for i in range(lenLk):    #比较Lk中的每一个元素和其他元素
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]    #取列表中的前面k-2个元素
            L1.sort(); L2.sort()
            if L1 == L2:    #如果两个集合的前面k-2个元素都相等，就将这两个集合合并成一个大小为k的集合
                retList.append(Lk[i]|Lk[j])
    return retList

def apriori(dataSet, minSupport = 0.5):
    '''
    生成候选项集的列表
    dataSet：数据集
    minSupport：最小支持度，默认为0.5
    Return:
    - L:候选项集列表
    - supportData：最频繁项集的支持度
    '''
    C1 = createC1(dataSet)    #创建C1
    D = list(map(set,dataSet))    #将数据集转化为集合列表
    L1, supportData = scanD(D,C1,minSupport)    #创建L1
    L = [L1]    #候选项集列表
    k =2    
    while(len(L[k-2])>0):   #创建更大项集的更大列表，直到下一个大的项集为空
        Ck = aprioriGen(L[k-2],k)    #创建Ck
        Lk, supK = scanD(D,Ck,minSupport)    #基于Ck来创建Lk
        supportData.update(supK)    #更新字典数据
        L.append(Lk)     #添加候选项集
        k += 1
    return L, supportData    #返回结果

In [11]:
L,supportData = apriori(dataSet)

In [12]:
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [13]:
L[0]

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [14]:
aprioriGen(L[0],2)

[frozenset({2, 5}),
 frozenset({3, 5}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({1, 3})]

In [15]:
L,supportData = apriori(dataSet,minSupport=0.7)
L

[[frozenset({5}), frozenset({2}), frozenset({3})], [frozenset({2, 5})], []]

In [18]:
def generateRules(L, supportData, minConf=0.7):
    '''
    关联规则生成函数。
    L ： 频繁项集列表
    supportData：频繁项集支持数据的字典
    minConf：最小可信度阈值，默认值为0.7
    Returns：
    - bigRuleList：包含可信度的规则列表
    '''
    bigRuleList = []
    for i in range(1,len(L)): #遍历L中的每一个频繁项集
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet] #创建只包含单个元素集合的列表H1
            if (i>1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)    #从包含两个或者更多元素的项集开始规则构建
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

def calcConf(freqSet, H, supportData, br1, minConf = 0.7):
    '''
    生成候选规则集合。
    freqSet ： 频繁项集
    H：可以出现在规则右部的元素列表
    supportData：频繁项集支持数据的字典
    br1：通过检查的bigRuleList
    minConf：最小可信度阈值，默认值为0.7
    Returns：
    - prunedH：包含可信度的规则列表
    '''
    prunedH = []    #创建一个空列表
    for conseq in H:   #遍历H中所有的项集
        conf = supportData[freqSet]/supportData[freqSet - conseq]    #计算可信度
        if conf >= minConf:
            print(freqSet-conseq,'-->',conseq,'conf',conf)
            br1.append((freqSet-conseq,conseq,conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, br1, minConf = 0.7):
    '''
    从最初的规则中生成更多的关联规则。
    freqSet ： 频繁项集
    H：可以出现在规则右部的元素列表
    supportData：频繁项集支持数据的字典
    br1：通过检查的bigRuleList
    minConf：最小可信度阈值，默认值为0.7
    Returns：
    - bigRuleList：包含可信度的规则列表
    '''
    m = len(H[0])    #计算H中的频繁集大小
    if (len(freqSet) > (m+1)):    #查看该频繁集是否大道可以移除大小为m的子集
        Hmp1 = aprioriGen(H,m+1)  #移除大小为m的子集，生成H中元素的无重复组合
        Hmp1 = calcConf(freqSet, Hmp1, supportData, br1, minConf)
        if(len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData,br1,minConf)

In [22]:
L,supportData = apriori(dataSet,minSupport=0.5)
rules = generateRules(L,supportData, minConf=0.7)
rules

frozenset({5}) --> frozenset({2}) conf 1.0
frozenset({2}) --> frozenset({5}) conf 1.0
frozenset({1}) --> frozenset({3}) conf 1.0


[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]

In [23]:
rules = generateRules(L,supportData, minConf=0.5)
rules


frozenset({3}) --> frozenset({2}) conf 0.6666666666666666
frozenset({2}) --> frozenset({3}) conf 0.6666666666666666
frozenset({5}) --> frozenset({3}) conf 0.6666666666666666
frozenset({3}) --> frozenset({5}) conf 0.6666666666666666
frozenset({5}) --> frozenset({2}) conf 1.0
frozenset({2}) --> frozenset({5}) conf 1.0
frozenset({3}) --> frozenset({1}) conf 0.6666666666666666
frozenset({1}) --> frozenset({3}) conf 1.0
frozenset({5}) --> frozenset({2, 3}) conf 0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf 0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf 0.6666666666666666


[(frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]

#### 示例：发现毒蘑菇的相似特征

In [24]:
mushDatSet = [line.split() for line in open('data/mushroom.dat').readlines()]

In [25]:
L,supportData = apriori(mushDatSet,minSupport=0.3)

In [26]:
for item in L[1]:
    if item.intersection('2'):print(item)

frozenset({'28', '2'})
frozenset({'2', '53'})
frozenset({'23', '2'})
frozenset({'2', '34'})
frozenset({'36', '2'})
frozenset({'59', '2'})
frozenset({'63', '2'})
frozenset({'67', '2'})
frozenset({'76', '2'})
frozenset({'2', '85'})
frozenset({'86', '2'})
frozenset({'90', '2'})
frozenset({'93', '2'})
frozenset({'39', '2'})


In [27]:
for item in L[3]:
    if item.intersection('2'):print(item)

frozenset({'59', '2', '28', '34'})
frozenset({'59', '2', '28', '85'})
frozenset({'59', '2', '28', '86'})
frozenset({'63', '2', '28', '34'})
frozenset({'63', '59', '2', '28'})
frozenset({'63', '2', '28', '85'})
frozenset({'63', '2', '28', '86'})
frozenset({'63', '2', '28', '39'})
frozenset({'2', '28', '85', '34'})
frozenset({'2', '28', '86', '34'})
frozenset({'2', '28', '86', '85'})
frozenset({'2', '28', '90', '34'})
frozenset({'59', '2', '28', '90'})
frozenset({'2', '28', '90', '85'})
frozenset({'2', '28', '90', '86'})
frozenset({'2', '28', '90', '39'})
frozenset({'2', '28', '90', '53'})
frozenset({'2', '28', '39', '34'})
frozenset({'59', '2', '28', '39'})
frozenset({'2', '28', '39', '85'})
frozenset({'2', '28', '39', '86'})
frozenset({'2', '28', '53', '34'})
frozenset({'2', '28', '53', '85'})
frozenset({'2', '28', '86', '53'})
frozenset({'2', '28', '39', '53'})
frozenset({'2', '53', '86', '34'})
frozenset({'2', '53', '86', '85'})
frozenset({'2', '90', '53', '34'})
frozenset({'2', '90'